In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from collections import OrderedDict
from typing import Tuple

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
print(train_images.shape)
print(train_labels.shape)
train_images = train_images / 255.0
test_images = test_images / 255.0

(60000, 28, 28)
(60000,)


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [7]:
def get_functional_model(learning_rate : float) -> tf.keras.Model:
    input_layer = tf.keras.layers.Input(shape = (28, 28, 1))
    conv_1 = tf.keras.layers.Conv2D(32, 5, use_bias = True, activation = 'relu')(input_layer)
    pool_1 = tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2)(conv_1)
    conv_2 = tf.keras.layers.Conv2D(64, 5, use_bias = True, activation = 'relu')(pool_1)
    pool_2 = tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2)(conv_2)
    conv_3 = tf.keras.layers.Conv2D(96, 3, use_bias = True, activation = 'relu')(pool_2)
    pool_3 = tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2)(conv_3)
    flat_1 = tf.keras.layers.Flatten()(pool_3)
    dense_out = tf.keras.layers.Dense(10, activation = 'softmax', name = "dense_last")(flat_1)
    
    model = tf.keras.models.Model(inputs = input_layer, outputs = dense_out)
    opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    
    model.compile(optimizer = opt, 
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy'])
    return model


learning_rate = 0.0003
batch_size = 10
epochs = 10
model : tf.keras.Model = get_functional_model(learning_rate)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 64)          51264     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 2, 2, 96)          5539

In [8]:
loss, acc = model.evaluate(test_images, test_labels)
print('Test accuracy : ', "{:0.2%}".format(acc))

313/313 [==============================] - 1s 4ms/step - loss: 2.3032 - accuracy: 0.0969
Test accuracy :  9.69%


In [9]:
train_log = model.fit(train_images, train_labels,
    batch_size = batch_size,
    epochs = epochs,
    validation_split = 0.1)

Epoch 1/10
5400/5400 [==============================] - 29s 5ms/step - loss: 0.5587 - accuracy: 0.7994 - val_loss: 0.4006 - val_accuracy: 0.8543
Epoch 2/10
5400/5400 [==============================] - 28s 5ms/step - loss: 0.3601 - accuracy: 0.8713 - val_loss: 0.3287 - val_accuracy: 0.8790
Epoch 3/10
5400/5400 [==============================] - 29s 5ms/step - loss: 0.3028 - accuracy: 0.8903 - val_loss: 0.3122 - val_accuracy: 0.8903
Epoch 4/10
5400/5400 [==============================] - 30s 6ms/step - loss: 0.2695 - accuracy: 0.9025 - val_loss: 0.2844 - val_accuracy: 0.8965
Epoch 5/10
5400/5400 [==============================] - 29s 5ms/step - loss: 0.2463 - accuracy: 0.9111 - val_loss: 0.2674 - val_accuracy: 0.9042
Epoch 6/10
5400/5400 [==============================] - 27s 5ms/step - loss: 0.2246 - accuracy: 0.9186 - val_loss: 0.2877 - val_accuracy: 0.8937
Epoch 7/10
5400/5400 [==============================] - 27s 5ms/step - loss: 0.2077 - accuracy: 0.9255 - val_loss: 0.2525 - val_ac

In [10]:
loss, acc = model.evaluate(test_images, test_labels)
print('Test accuracy : ', "{:0.2%}".format(acc))

313/313 [==============================] - 1s 4ms/step - loss: 0.2745 - accuracy: 0.9075
Test accuracy :  90.75%


In [11]:
save_dir = "./logs/"
save_path = save_dir + "model_official_1"
# model.save(save_path)

INFO:tensorflow:Assets written to: ./logs/model_official_1\assets


INFO:tensorflow:Assets written to: ./logs/model_official_1\assets


In [12]:
# Load model_v3
save_dir = "./logs/"
save_path = save_dir + "model_v3"
# model = tf.keras.models.load_model(save_path)
loss, acc = model.evaluate(test_images, test_labels)
print('Test accuracy : ', "{:0.2%}".format(acc))

313/313 [==============================] - 1s 4ms/step - loss: 0.2745 - accuracy: 0.9075
Test accuracy :  90.75%


In [15]:
q_aware_model = tfmot.quantization.keras.quantize_model(model)
q_aware_model.compile(optimizer = 'adam', 
    loss = 'sparse_categorical_crossentropy', 
    metrics = ['accuracy'])
q_aware_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 quantize_layer_2 (QuantizeL  (None, 28, 28, 1)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_9 (QuantizeWra  (None, 24, 24, 32)       899       
 pperV2)                                                         
                                                                 
 quant_max_pooling2d_9 (Quan  (None, 12, 12, 32)       1         
 tizeWrapperV2)                                                  
                                                                 
 quant_conv2d_10 (QuantizeWr  (None, 8, 8, 64)         51395     
 apperV2)                                                  

In [16]:
q_aware_test_loss, q_aware_test_acc = q_aware_model.evaluate(test_images, test_labels)
print('Test accuracy : ', "{:0.2%}".format(q_aware_test_acc))

313/313 [==============================] - 2s 6ms/step - loss: 4.0996 - accuracy: 0.1000
Test accuracy :  10.00%


In [17]:
train_log = q_aware_model.fit(train_images, train_labels,
    batch_size = 128,
    # epochs = 15,
    epochs = 1,
    validation_split = 0.1)

422/422 [==============================] - 20s 45ms/step - loss: 0.1482 - accuracy: 0.9462 - val_loss: 0.2671 - val_accuracy: 0.9122


In [18]:
q_aware_test_loss, q_aware_test_acc = q_aware_model.evaluate(test_images, test_labels)
print('Test accuracy : ', "{:0.2%}".format(q_aware_test_acc))

313/313 [==============================] - 1s 5ms/step - loss: 0.2876 - accuracy: 0.9046
Test accuracy :  90.46%


In [188]:
# Save quantized model
save_dir = "./logs/"
save_path = save_dir + "model_q_official_1"
# q_aware_model.save(save_path)

INFO:tensorflow:Assets written to: ./logs/model_q_official_1\assets


INFO:tensorflow:Assets written to: ./logs/model_q_official_1\assets


In [312]:
# Load model 
save_dir = "./logs/"
save_path = save_dir + "model_q_official_1"
q_aware_model : tf.keras.Model
# with tfmot.quantization.keras.quantize_scope():
#     q_aware_model = tf.keras.models.load_model(save_path)
q_aware_test_loss, q_aware_test_acc = q_aware_model.evaluate(test_images, test_labels)
print('Test accuracy : ', "{:0.2%}".format(q_aware_test_acc))

313/313 [==============================] - 4s 11ms/step - loss: 0.2876 - accuracy: 0.9046
Test accuracy :  90.46%


In [313]:
for i in range(len(q_aware_model.layers)):
    print("Layer : ", i, q_aware_model.layers[i].name," - params : ", len(q_aware_model.layers[i].variables))#, len(q_aware_model.layers[i]), "Weights len")

Layer :  0 input_4  - params :  0
Layer :  1 quantize_layer_2  - params :  3
Layer :  2 quant_conv2d_9  - params :  7
Layer :  3 quant_max_pooling2d_9  - params :  1
Layer :  4 quant_conv2d_10  - params :  7
Layer :  5 quant_max_pooling2d_10  - params :  1
Layer :  6 quant_conv2d_11  - params :  7
Layer :  7 quant_max_pooling2d_11  - params :  1
Layer :  8 quant_flatten_3  - params :  1
Layer :  9 quant_dense_last  - params :  7


In [40]:
bit_width = 8
quantized_and_dequantized = OrderedDict()
quantized = OrderedDict()
new_quantized_and_dequantized = OrderedDict()
new_quantized = OrderedDict()
layer_index_list = []
keys_list = []

layer : tfmot.quantization.keras.QuantizeWrapperV2
for i, layer in enumerate(q_aware_model.layers):
    quantizer : tfmot.quantization.keras.quantizers.Quantizer
    weight : tf.Variable
    if hasattr(layer, '_weight_vars'):
        for weight, quantizer, quantizer_vars in layer._weight_vars:
            min_var = quantizer_vars['min_var']
            max_var = quantizer_vars['max_var']

            key = weight.name[:-2]
            layer_index_list.append(i)
            keys_list.append(key)
            quantized_and_dequantized[key] = quantizer(inputs = weight, training = False, weights = quantizer_vars)
            quantized[key] = np.round(quantized_and_dequantized[key] / max_var * (2**(bit_width-1)-1))

            if "conv2d" in layer.name:
                new_quantized_and_dequantized[key] = tf.quantization.fake_quant_with_min_max_vars_per_channel(weight, min_var, max_var, bit_width, narrow_range = True, name = "New_quantized_" + str(i))
                new_quantized[key] = np.round(new_quantized_and_dequantized[key] / max_var * (2**(bit_width-1)-1))
            elif "dense" in layer.name:
                new_quantized_and_dequantized[key] = tf.quantization.fake_quant_with_min_max_vars(weight, min_var, max_var, bit_width, narrow_range = True, name = "New_quantized_" + str(i))
                new_quantized[key] = np.round(new_quantized_and_dequantized[key] / max_var * (2**(bit_width-1)-1))

for key in quantized:
    # print("Fake Quantized")
    print(key)
    if "dense" not in key:
        # print(quantized_and_dequantized[key][:,:,0,0])
        print(quantized[key][:,:,0,0])
    else:
        # print(quantized_and_dequantized[key][:,0])
        print(quantized[key][:,0])

    # print("New Fake Quantized")
    # print(key)
    # if "dense" not in key:
    #     print(new_quantized_and_dequantized[key][:,:,0,0])
    #     print(new_quantized[key][:,:,0,0])
    # else:
    #     print(new_quantized_and_dequantized[key][:,0])
    #     print(new_quantized[key][:,0])

conv2d/kernel
[[ -34.    9.   41.  -66. -127.]
 [ -70.    8.   57.   27.  -73.]
 [   5.   39.  -30.  -17.   33.]
 [  47.  -23.   -1.  -23.   -1.]
 [  39.   16.  -19.    4.   12.]]
conv2d_1/kernel
[[-18. -40. -32.   4.  -3.]
 [  5. -28. -42.  12.  20.]
 [ 47. -42. -14. -11.  32.]
 [ 41. -15.   0. -15.  17.]
 [ 35. -10.  -4.  17.  25.]]
conv2d_2/kernel
[[ 69.  51.  -2.]
 [  2. -98. -63.]
 [ 67.   5. -44.]]
dense_last/kernel
[  4.   1.  -6.   6.  -1. -10. -18.  20. -18.  -8.  -1.  -6. -30. -13.
  -8.  11.  13.   5. -35. -12. -24. -18.   8.   8.  -4. -27.  18.  15.
 -46. -23.  21. -42.  19.  12.  12.  -1.   9.  -8.   1.  18.  -6.  13.
  -2.  13.   8. -10.  12.  27.   5.  -1.  -5.  -5. -11.  13. -29. -25.
  -8.   4.  19.   4. -19. -26.  32.  15. -35. -12.   0.  31.  -8. -25.
  10.  21.  14.  14.  -9.   7. -24. -23. -15.  18.   6.  27.  26.  13.
   9.  34.   6. -25. -23. -14. -42. -17.  -2.   7. -18.   9.]


In [315]:
def self_quantize_function(input, min_var, max_var, bits, narrow_range = False):
    if not narrow_range:
        scale = (max_var - min_var) / (2**bits - 1)
    else:
        scale = (max_var - min_var) / (2**bits - 2)
    # min_adj = scale * np.round(min_var / scale)
    # max_adj = max_var + min_adj - min_var
    # print("Scale : ", scale)
    return scale * np.round(input / scale)

for idx, layer_index in enumerate(layer_index_list):
    m_vars = {variable.name: variable for i, variable in enumerate(q_aware_model.layers[layer_index].non_trainable_variables) if keys_list[idx] in variable.name}
    kernel = {variable.name: variable for i, variable in enumerate(q_aware_model.layers[layer_index].trainable_variables) if "kernel" in variable.name}
    min_key = list(key for key in m_vars if "min" in key)[0]
    max_key = list(key for key in m_vars if "max" in key)[0]
    min_var = m_vars[min_key]
    max_var = m_vars[max_key]

    kernel_index = 0
    self_quantized_and_dequantized = self_quantize_function(q_aware_model.layers[layer_index].trainable_variables[kernel_index], min_var, max_var, bit_width, narrow_range = True)
    
    print(keys_list[idx])
    # print("Self Quantized")
    if "dense" not in keys_list[idx]:
        # print(self_quantized_and_dequantized[:,:,0,0])
        self_quantized = np.round(self_quantized_and_dequantized / max_var * (2**(bit_width - 1) - 1))
        print(self_quantized[:,:,0,0])
    else:
        # print(self_quantized_and_dequantized[:,0])
        self_quantized = np.round(self_quantized_and_dequantized / max_var * (2**(bit_width - 1) - 1))
        print(self_quantized[:,0])

conv2d_9/kernel
[[ -20.   67.   38.  127.   26.]
 [  22.    1.  -76.   67. -103.]
 [  39.   -7.  -97.   51.  -54.]
 [ -42.    9.   59.  -26.  -40.]
 [  23.  -89.  -15.   94.    4.]]
conv2d_10/kernel
[[-12.   4.  25.   5. -62.]
 [-13.   8. -11.   4.  12.]
 [ 21.  20.  12.   2.   2.]
 [ 30.  16.   6.  41. -32.]
 [ 22.  24.  10.  27. -25.]]
conv2d_11/kernel
[[ 29. -45.  47.]
 [ -3. -35.  30.]
 [ -8.   4.  26.]]
dense_last/kernel
[-22.  -8. -25.  -7.  23. -14.   4. -52. -11. -28.   9.   0.  -2.  24.
  16. -36.  40. -24.  41.  29. -24.  10.  14.  26. -19. -23.   1. -14.
  -6.  32.  39. -39. -20. -46. -27. -20.  33. -45. -24.  -5. -47.  33.
 -52. -29. -28. -26. -32. -31.   6.  -7.   4.  36. -23.   4. -10. -33.
 -29.  23.   4. -14.  22.  37. -32. -34. -48.   1. -32.  19. -17. -37.
 -10. -55. -40. -19.  25. -13. -16.  21. -32.   5. -11.  19. -48.   2.
  30.   1.  26. -29.  50.  34.  21.  -8.  29.  -3.  37. -25.]


In [24]:
l = 2
print(q_aware_model.layers[l].quantize_config.get_config())
print(q_aware_model.layers[l].quantize_config.activation_quantizer)

{'weight_attrs': ['kernel'], 'activation_attrs': ['activation'], 'quantize_output': False}


In [316]:
# Conversion to TF Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\rosal\AppData\Local\Temp\tmpyawuma0f\assets


INFO:tensorflow:Assets written to: C:\Users\rosal\AppData\Local\Temp\tmpyawuma0f\assets
c:\Users\rosal\py_master_project\lib\site-packages\tensorflow\lite\python\convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [317]:
def evaluate_model(interpreter: tf.lite.Interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    # print("Shape : ", test_image.shape)
    test_image = np.expand_dims(test_image, axis = 0).astype(np.float32)
    test_image = np.expand_dims(test_image, axis = 3).astype(np.float32)
    # print("New Shape : ", test_image.shape)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [318]:
interpreter = tf.lite.Interpreter(model_content = quantized_tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape: ", input_details[0]['shape'])
print("Output Shape: ", output_details[0]['shape'])

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_test_acc)

Input Shape:  [ 1 28 28  1]
Output Shape:  [ 1 10]
Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9046
Quant TF test accuracy: 0.9046000242233276


In [319]:
save_dir = "./logs/"
quant_file = 'quant_model_q_official_1.tflite'
save_path = save_dir + quant_file
# with open(save_path, 'wb') as f:
#   f.write(quantized_tflite_model)

In [320]:
def prob_mass_gen(bits : int):
    """ Discrete triangular distribution
        - Only a max probability and slope are allowed """
    values = np.arange(0, bits)
    n = len(values)
    p_max = 2/n
    m_max = p_max/(n - 1)
    # Probability calculation
    p = p_max * 1.0
    m = 2/(n - 1)*(p- 1/n)
    probabilities = [p + m*(i - values[-1]) for i in values]
    return values, probabilities

def random_bit_flipper(value : int):
    r""" All values are in 8 bits, MSB have higher probability of getting flipped
        - It is assumed value is a signed 8 bit number """
    bits, probs = prob_mass_gen(8)
    bit_pos = np.random.choice(bits, p = probs)

    # print("Value", value, bin(value))

    # Negative 2 Complement conversion
    if value < 0:
        value = (-value ^ 0xFF) + 1

    # print("Value after 2 complement conversion:", value, bin(value))

    flip_mask = 1 << bit_pos
    flipped = value ^ flip_mask
    # print("Bit to be flipped", bit_pos)
    # print("Flipped value", flipped, bin(flipped))

    # Negative back conversion 2 Complement
    if flipped >= 128:
        flipped = -((flipped ^ 0xFF) + 1)
        # print("Negative back conversion", flipped)
    return flipped

val = np.random.randint(-127,127)
print(val)
print(random_bit_flipper(val))

-66
-2


In [324]:
# Load model 
save_dir = "./logs/"
save_path = save_dir + "model_q_official_1"
q_aware_copy : tf.keras.Model
with tfmot.quantization.keras.quantize_scope():
    q_aware_copy = tf.keras.models.load_model(save_path)

# random_channel = np.random.randint(0,32)
# random_kernel_x = np.random.randint(0,5)
# random_kernel_y = np.random.randint(0,5)
random_channel = 0
random_kernel_x = 0
random_kernel_y = 0
idx = 0
key = keys_list[idx]
layer_index = layer_index_list[idx]
kernel_index = 0
print("Random position", random_kernel_x, random_kernel_y, 0, random_channel)
print(new_quantized[key][:,:,0,random_channel])
print(new_quantized[key][random_kernel_x,random_kernel_y,0,random_channel])
print(q_aware_copy.layers[layer_index].trainable_variables[kernel_index][:,:,0,random_channel])
print(q_aware_copy.layers[layer_index].trainable_variables[kernel_index][random_kernel_x,random_kernel_y,0,random_channel])

flipped_kernel_value = random_bit_flipper(int(new_quantized[key][random_kernel_x,random_kernel_y,0,random_channel]))
print(flipped_kernel_value)

Random position 0 0 0 0
[[ -20.   67.   38.  127.   26.]
 [  22.    1.  -76.   67. -103.]
 [  39.   -7.  -97.   51.  -54.]
 [ -42.    9.   59.  -26.  -40.]
 [  23.  -89.  -15.   94.    4.]]
-20.0
tf.Tensor(
[[-0.03571794  0.12105308  0.0683602   0.22875616  0.04659204]
 [ 0.03975419  0.00167603 -0.13742748  0.12099734 -0.18465865]
 [ 0.0703144  -0.01184483 -0.17510103  0.09140633 -0.09658822]
 [-0.07492749  0.01618467  0.10540633 -0.04606877 -0.07135177]
 [ 0.04229205 -0.16009727 -0.02699903  0.16959691  0.0072944 ]], shape=(5, 5), dtype=float32)
tf.Tensor(-0.03571794, shape=(), dtype=float32)
108


In [325]:
print(keys_list)
# idx = key_index_list.index("conv2d_9/kernel")
idx = 0
layer_index = layer_index_list[idx]
kernel_index = 0
m_vars = {variable.name: variable for i, variable in enumerate(q_aware_model.layers[layer_index].non_trainable_variables) if keys_list[idx] in variable.name}
kernel = {variable.name: variable for i, variable in enumerate(q_aware_model.layers[layer_index].trainable_variables) if "kernel" in variable.name}
min_key = list(key for key in m_vars if "min" in key)[0]
max_key = list(key for key in m_vars if "max" in key)[0]
min_var = m_vars[min_key]
max_var = m_vars[max_key]

print(flipped_kernel_value)
self_flipped = flipped_kernel_value * max_var.numpy()[random_channel] / (2**(bit_width - 1) - 1)
print(self_flipped)
original_val = q_aware_copy.layers[layer_index].trainable_variables[kernel_index][random_kernel_x,random_kernel_y,0,random_channel]
print(original_val)
update_kernel = q_aware_copy.layers[layer_index].trainable_variables[kernel_index].numpy()
update_kernel[random_kernel_x,random_kernel_y,0,random_channel] = self_flipped
# update_kernel[random_kernel_x,random_kernel_y,0,random_channel] = original_val.numpy()
print(hex(id(q_aware_copy.layers[layer_index].trainable_variables[kernel_index])))
q_aware_copy.layers[layer_index].trainable_variables[kernel_index].assign(update_kernel)
print(q_aware_copy.layers[layer_index].trainable_variables[kernel_index][:,:,0,random_channel])
print(hex(id(q_aware_copy.layers[layer_index].trainable_variables[kernel_index])))


['conv2d_9/kernel', 'conv2d_10/kernel', 'conv2d_11/kernel', 'dense_last/kernel']
108
0.19451098883245874
tf.Tensor(-0.03571794, shape=(), dtype=float32)
0x1f9fa3a5540
tf.Tensor(
[[ 0.19451098  0.12105308  0.0683602   0.22875616  0.04659204]
 [ 0.03975419  0.00167603 -0.13742748  0.12099734 -0.18465865]
 [ 0.0703144  -0.01184483 -0.17510103  0.09140633 -0.09658822]
 [-0.07492749  0.01618467  0.10540633 -0.04606877 -0.07135177]
 [ 0.04229205 -0.16009727 -0.02699903  0.16959691  0.0072944 ]], shape=(5, 5), dtype=float32)
0x1f9fa3a5540


In [326]:
# Conversion to TF Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_copy)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
q_copy_tflite_model = converter.convert()

save_dir = "./logs/"
quant_file = 'quant_model_q_perturbed_1.tflite'
save_path = save_dir + quant_file
with open(save_path, 'wb') as f:
  f.write(q_copy_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\rosal\AppData\Local\Temp\tmptiq_mtw0\assets


INFO:tensorflow:Assets written to: C:\Users\rosal\AppData\Local\Temp\tmptiq_mtw0\assets
c:\Users\rosal\py_master_project\lib\site-packages\tensorflow\lite\python\convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [327]:
interpreter = tf.lite.Interpreter(model_content = q_copy_tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape: ", input_details[0]['shape'])
print("Output Shape: ", output_details[0]['shape'])

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_test_acc)

Input Shape:  [ 1 28 28  1]
Output Shape:  [ 1 10]
Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9025
Quant TF test accuracy: 0.9046000242233276


In [41]:
import ast
LOAD_PATH_Q_AWARE = "./model/" + "model_q_aware_final_01"

q_aware_model : tf.keras.Model
with tfmot.quantization.keras.quantize_scope():
    q_aware_model = tf.keras.models.load_model(LOAD_PATH_Q_AWARE)


def bit_flipper(value : int, bit_pos : int) -> int:
    """ Random bit flipper 
    -
    Obtains a value and bit position and flips it.
    - All values are in 8 bits, MSB have higher probability of getting flipped
    - It is assumed value is a signed 8 bit number """
    # Negative 2 Complement conversion
    if value < 0:
        value = (-value ^ 0xFF) + 1
    flip_mask = 1 << bit_pos
    flipped_value = value ^ flip_mask
    # Negative back conversion 2 Complement
    if flipped_value >= 128:
        flipped_value = -((flipped_value ^ 0xFF) + 1)
    return flipped_value

In [47]:
df = pd.read_csv('Performance.csv')
position_disrupted_list = []
bit_flipped_list = []
for d in df['position_disrupted']:
    try:
        position_disrupted_list.append(ast.literal_eval(d))
    except:
        position_disrupted_list.append(tuple())
for d in df['bit_disrupted']:
    try:
        bit_flipped_list.append(int(d))
    except:
        bit_flipped_list.append(-1)

out_list = []
conv_idx = 0 
bit_width = 8
key = keys_list[conv_idx]
layer_index = layer_index_list[conv_idx]
T_VARIABLES_KERNEL_INDEX = 0

for i, position in enumerate(position_disrupted_list):
    entry = {}
    entry['data'] = position
    if len(position) > 1:
        m_vars = {variable.name: variable for i, variable in enumerate(q_aware_model.layers[layer_index].non_trainable_variables) if keys_list[conv_idx] in variable.name}
        min_key = list(key for key in m_vars if "min" in key)[0]
        max_key = list(key for key in m_vars if "max" in key)[0]
        min_var = m_vars[min_key]
        max_var = m_vars[max_key]
        entry['min_var'] = min_var[position[-1]].numpy()
        entry['max_var'] = max_var[position[-1]].numpy()
        entry['original_weight_value'] = q_aware_model.layers[layer_index].trainable_variables[T_VARIABLES_KERNEL_INDEX][position].numpy()
        entry['quantized_value'] = quantized[key][position]
        entry['flipped_quantized_value'] = bit_flipper(int(quantized[key][position]), bit_flipped_list[i])
        entry['flipped_weight_value'] = entry['flipped_quantized_value'] * max_var.numpy()[position[-1]] / (2**(bit_width - 1) - 1)
    else:
        entry['min_var'] = None
        entry['max_var'] = None
        entry['original_weight_value'] = None
        entry['quantized_value'] = None
        entry['flipped_quantized_value'] = None
        entry['flipped_weight_value'] = None
    out_list.append(entry)
out_df = pd.DataFrame(out_list)
out_df.to_csv('Max_Min.csv')